In [56]:
import Pkg
Pkg.activate("MMLDS_project")

#SINDy
#Pkg.add("DataDrivenDiffEq")
#Pkg.add("DataDrivenSparse")
#Pkg.add("Subscripts")

#RESERVOIR 
#Pkg.update()
#Pkg.instantiate()
#Pkg.add("ProgressMeter")
#Pkg.add("DynamicalSystems")
#Pkg.add("ReservoirComputing")
#Pkg.add("Plots")
#Pkg.add("Printf")
#Pkg.add("DifferentialEquations")
#Pkg.add("OrdinaryDiffEq")
#Pkg.add("NetCDF")
#Pkg.add("JLD2")

#SINDy
using DataDrivenDiffEq
using DataDrivenSparse
using Subscripts #not working yet, took part of the code

#RESERVOIR
using ProgressMeter
using DynamicalSystems, ReservoirComputing, Plots, Printf
using DifferentialEquations
using OrdinaryDiffEq
using NetCDF
using JLD2

  Activating project at `~/Documents/GitHub/compare_ML_DynamicalSystem/MMLDS_project/src/MMLDS_project`


In [57]:
include("data.jl")
include("SINDy.jl")

train_SINDy (generic function with 3 methods)

### Helper functions

In [73]:
#using Subscripts Package is not working, props gehen raus an https://github.com/HaoxuanGuo/Subscripts.jl/blob/main/src/Subscripts.jl
super_chars = ['⁰', '¹', '²', '³', '⁴', '⁵', '⁶', '⁷', '⁸', '⁹']
sub_chars = ['₀', '₁', '₂', '₃', '₄', '₅', '₆', '₇', '₈', '₉']

function sub(s)
    res = ""
    for c in s
        if c >= '0' && c <= '9'
            res *= sub_chars[c - '0' + 1]
        end
    end
    return res
end

function recover(s::String)
    res = ""
    for c in s
        for i in 1:length(super_chars)
            if c == super_chars[i] || c == sub_chars[i]
                res *= '0' + i - 1
            end
        end
    end
    return res
end

recover (generic function with 1 method)

In [58]:
#Loading and changing El Nino Data
path_to_data = "sst.mon.mean.nc"

x = ncread(path_to_data, "sst") #sst = Sea Surface Temperature
x_reduced = x[121:170, 86:95, :] #only concerned with the important region for the El Nino happening

X_vector = compress_data_matrix(x_reduced, 5, false, true)

20×2040 Matrix{Float64}:
 28.0325  27.8742  28.1076  28.688   …  28.621   29.4195  29.5172  29.0083
 28.2426  28.2316  28.488   28.9117     28.3489  29.3835  29.6865  29.3908
 28.5179  28.6131  28.8473  29.1355     28.6434  29.723   29.7411  29.3795
 28.84    28.8382  29.0742  29.1841     29.4347  30.2484  29.8994  29.4668
 29.023   28.9616  29.1702  29.2372     29.4146  30.2277  29.8914  29.5535
 29.1572  29.1256  29.2598  29.2776  …  29.7037  30.4001  30.031   29.7284
 29.1264  29.117   29.196   29.2381     29.8099  30.4189  30.0984  29.8489
 28.9993  29.0324  29.076   29.1209     30.1048  30.5088  30.2152  29.9283
 28.7191  28.7542  28.6979  28.8395     30.32    30.6631  30.3864  30.0508
 28.4407  28.3725  28.2181  28.6071     30.32    30.5643  30.4019  30.1978
 28.7257  28.5573  28.5916  28.9932  …  27.2255  28.4254  29.3495  29.8018
 28.7958  28.7403  28.7715  29.102      26.6425  28.0568  29.141   29.8132
 28.8938  28.9374  28.9322  29.1185     27.5829  28.7421  29.3249  29.7972


### test cases

#### learn only on one coordinate
#### use more basis functions
#### SINDy dependent on scaling factors?

In [108]:
function train_SINDy(ode_sol, n, threshold=1e-1, λ=1e-1)
    
    ddprob = DataDrivenProblem(ode_sol)
    
    @variables t x(t) y(t) z(t)  # Symbolic variables
    u = [x, y, z]
    #basis = Basis([polynomial_basis(u, n),sin_basis(u, 4), cos_basis(u, 4)], u, iv = t)
    basis = Basis(sin_basis(u, 4), u, iv = t)
     
    optimiser = STLSQ(threshold, λ) #sparsity cut off threshold, Ridge regression parameter

    ddsol = solve(ddprob, basis, optimiser, options = DataDrivenCommonOptions(digits = 2))
    println(get_basis(ddsol))

    return ddsol

end

train_SINDy (generic function with 3 methods)

In [109]:
#shrink the dimensions to see if it is working
#we have to make more basis variables
x_sindy = X_vector[1:3,:]
ddsol = train_SINDy(x_sindy, 4, 1e-1, 0)

ddsol.prob.p[1]


Model ##Basis#375 with 3 equations
States : x(t) y(t) z(t)
Parameters : 36
Independent variable: t
Equations
Difference(t; dt=1.0, update=false)(x(t)) = p₁₀*sin(4x(t)) + p₁₁*sin(4y(t)) + p₁₂*sin(4z(t)) + p₁*sin(x(t)) + p₂*sin(y(t)) + p₃*sin(z(t)) + p₄*sin(2x(t)) + p₇*sin(3x(t)) + p₅*sin(2y(t)) + p₆*sin(2z(t)) + p₈*sin(3y(t)) + p₉*sin(3z(t))
Difference(t; dt=1.0, update=false)(y(t)) = p₁₆*sin(2x(t)) + p₁₉*sin(3x(t)) + p₁₃*sin(x(t)) + p₁₄*sin(y(t)) + p₁₇*sin(2y(t)) + p₂₀*sin(3y(t)) + p₁₅*sin(z(t)) + p₂₂*sin(4x(t)) + p₂₃*sin(4y(t)) + p₁₈*sin(2z(t)) + p₂₁*sin(3z(t)) + p₂₄*sin(4z(t))
Difference(t; dt=1.0, update=false)(z(t)) = p₂₈*sin(2x(t)) + p₂₅*sin(x(t)) + p₃₁*sin(3x(t)) + p₃₄*sin(4x(t)) + p₂₉*sin(2y(t)) + p₃₂*sin(3y(t)) + p₃₅*sin(4y(t)) + p₃₀*sin(2z(t)) + p₃₃*sin(3z(t)) + p₃₆*sin(4z(t)) + p₂₆*sin(y(t)) + p₂₇*sin(z(t))



-13.08

In [112]:
#this has to be done by hand right now

x,y,z = ddsol.basis
#print(x)
x_truncated = replace(string(y), "(t)" => "")
x_truncated = replace(x_truncated, "Difference(t; dt=1.0, update=false)" => "")
x_truncated = replace(x_truncated, "~" => "=")
#x_truncated = replace(x_truncated, "₁" => "["*string(1)*"]")
for i in length(ddsol.prob.p):-1:1
    number = string(i)
    sub_number = sub(number)
    s = "["*string(i)*"]"
    x_truncated = replace(x_truncated, sub_number => "[" * string(i) * "]")
end
print(x_truncated)

(y) = p[16]*sin(2x) + p[19]*sin(3x) + p[13]*sin(x) + p[14]*sin(y) + p[17]*sin(2y) + p[20]*sin(3y) + p[15]*sin(z) + p[22]*sin(4x) + p[23]*sin(4y) + p[18]*sin(2z) + p[21]*sin(3z) + p[24]*sin(4z)

### Last step to this one has to be done by hand for every equation for now

In [117]:
#create diffeq from SINDy output
function SINDy_found_function(u,p,t)
    x, y, z = u
    dx = p[10]*sin(4x) + p[11]*sin(4y) + p[12]*sin(4z) + p[1]*sin(x) + p[2]*sin(y) + p[3]*sin(z) + p[4]*sin(2x) + p[7]*sin(3x) + p[5]*sin(2y) + p[6]*sin(2z) + p[8]*sin(3y) + p[9]*sin(3z)    
    dy = p[16]*sin(2x) + p[19]*sin(3x) + p[13]*sin(x) + p[14]*sin(y) + p[17]*sin(2y) + p[20]*sin(3y) + p[15]*sin(z) + p[22]*sin(4x) + p[23]*sin(4y) + p[18]*sin(2z) + p[21]*sin(3z) + p[24]*sin(4z)    
    dz = p[28]*sin(2x) + p[25]*sin(x) + p[31]*sin(3x) + p[34]*sin(4x) + p[29]*sin(2y) + p[32]*sin(3y) + p[35]*sin(4y) + p[30]*sin(2z) + p[33]*sin(3z) + p[36]*sin(4z) + p[26]*sin(y) + p[27]*sin(z)    
    return [dx, dy, dz]
end

function create_SINDy_trajectory(x0 = [1.,1.,1.],dt = 1, tstart = 1., tend = 100.)

    p_SINDy = ddsol.prob.p
    tspan = (tstart, tend)
    saveat = tstart:dt:tend
    x0 = X_vector[1:3,1]

    prob = ODEProblem(lorenz, x0, tspan, p_SINDy) 
    sol = solve(prob, Tsit5(), saveat=saveat)

    return sol
end

sol = create_SINDy_trajectory()
print(sol)


┌ Warning: dt(1.4210854715202004e-14) <= dtmin(1.4210854715202004e-14) at t=1.3069985477944173. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /home/tom/.julia/packages/SciMLBase/gTrkJ/src/integrator_interface.jl:529


ODESolution{Float64, 2, Vector{Vector{Float64}}, Nothing, Nothing, Vector{Float64}, Vector{Vector{Vector{Float64}}}, ODEProblem{false,Vector{Float64},Tuple{Float64, Float64},…}, Tsit5{Static.False,…}, OrdinaryDiffEq.InterpolationData{

ODEFunction{false,SciMLBase.AutoSpecialize,…}, Vector{Vector{Float64}}, Vector{Float64}, Vector{Vector{Vector{Float64}}}, OrdinaryDiffEq.Tsit5ConstantCache}, DiffEqBase.DEStats, Nothing}(

[[28.032480239868164, 28.24264144897461, 28.517921447753906]], nothing, nothing, [1.0], 

[[[28.032480239868164, 28.24264144897461, 28.517921447753906]]], 

...

.StandardODEProblem()), Tsit5(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false)

), 

OrdinaryDiffEq.InterpolationData{ODEFunction{false,SciMLBase.AutoSpecialize,…}, Vector{Vector{Float64}}, Vector{Float64}, Vector{Vector{Vector{Float64}}}, OrdinaryDiffEq.Tsit5ConstantCache}(ODEFunction{false,SciMLBase.AutoSpecialize,…}(lorenz, LinearAlgebra.UniformScaling{Bool}(true), nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, SciMLBase.DEFAULT_OBSERVED, nothing, nothing), [[28.032480239868164, 28.24264144897461, 28.517921447753906]], [1.0], [[[28.032480239868164, 28.24264144897461, 28.517921447753906]]], false, 

OrdinaryDiffEq.Tsit5ConstantCache()), false, 0, DiffEqBase.DEStats


Number of function 1 evaluations:                  2074005
Number of function 2 evaluations:                  0
Number of W matrix evaluations:                    0
Number of linear solves:                           0
Number of Jacobians created:                       0
Number of nonlinear solver iterations:             0
Number of nonlinear solver convergence failures:   0
Number of rootfind condition calls:                0
Number of accepted steps:                          345553
Number of rejected steps:                          114, nothing, SciMLBase.ReturnCode.DtLessThanMin)

In [116]:
sol[1,1:100]

BoundsError: BoundsError: attempt to access 1-element Vector{Vector{Float64}} at index [2]

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.


In [118]:
savePath = "/home/tom/Documents/GitHub/compare_ML_DynamicalSystem/MMLDS_project/Plots/SINDy/"

for i in 1:3
    i_data = X_vector[i, 1:100]
    p = plot(i_data, idxs=[1], tspan=(0, 100), label="El Nino Data", title="on real world data")
    p = plot!(sol[i,1:100], idxs=[1], tspan=(0, 100), label="SINDy")
    saveEnding = string(i)
    savePath = savePath* string(saveEnding)*".png" 
    savefig(p, savePath)
end


BoundsError: BoundsError: attempt to access 1-element Vector{Vector{Float64}} at index [2]

Some of the types have been truncated in the stacktrace for improved reading. To emit complete information
in the stack trace, evaluate `TruncatedStacktraces.VERBOSE[] = true` and re-run the code.
